In [1]:
import torch


from tqdm.notebook import tqdm

from neurovlm.text_input import search_papers_from_text, search_wiki_from_text, generate_llm_response_from_text, search_cogatlas_from_text
from neurovlm.brain_input import (
    search_papers_from_brain,
    generate_llm_response_from_brain,
    resample_networks_to_mask
)

from neurovlm.retrieval_resources import (
    _load_autoencoder,
    _load_networks,
    _load_masker )

## Ollama Setup

** 1. Install Ollama **
- ** using brew: **
```sh
brew install ollama
```

** 2. Start Ollama Service **
```
ollama serve
```

** 3. Download the ollama app **

** 4. Pull a Model(Our default model is qwen2.5:7b) **
```
ollama pull qwen2.5:7b-instruct  or ollama pull qwen2.5:3b-instruct
```


## Load data and models

In [2]:
# Load data and Specter
autoencoder = _load_autoencoder()

masker = _load_masker()

## Example query for text

The titles, abstract, cognitive concepts, cognitive tasks, and cognitive disorders most related to the query will be passed to the language model.

The function accepts a string input; no need to encode the input text before passing it in

In [3]:
query = "default mode network"

In [4]:
query

'default mode network'

In [5]:
# Get the top k similar papers
abstract, titles = search_papers_from_text(query, show_titles = True)

There are adapters available but none are activated for the forward pass.


Top matches:
1. An information-theoretic analysis of resting-state versus task fMRI
2. Altered default mode network causal connectivity patterns in autism spectrum disorder revealed by Liang information flow analysis
3. Default mode network connectivity distinguishes chemotherapy-treated breast cancer survivors from controls.
4. The effect of seed location on functional connectivity: evidence from an image-based meta-analysis
5. Exploring the neural substrates of misinformation processing.


In [6]:
context, titles = search_wiki_from_text("dementia", show_titles = True)

Top matches:
1. Idiopathic disease
2. SLC10A2


In [7]:
definition, terms, _ = search_cogatlas_from_text(query, show_titles=True)

Top matches:
1. declarative knowledge
2. spreading activation
3. inference
4. active maintenance
5. intertemporal choice


In [8]:
definition, terms, _ = search_cogatlas_from_text(query, show_titles=True, category= "cogatlas_disorder")

Top matches:
1. mild cognitive impairment
2. impotence
3. melancholia
4. anosognosia
5. pedophilia


In [9]:
definition, terms, _ = search_cogatlas_from_text(query, show_titles=True, category= "cogatlas_task")

Top matches:
1. lateral facilitation
2. autism spectrum quotient
3. autobiographical memory task
4. local computation
5. theory of mind task


In [ ]:
output = generate_llm_response_from_text(query)

In [12]:
print(output)

### Overview

The publications related to the user query "default mode network" suggest that the default mode network (DMN) is involved in processes associated with resting activity, such as autobiographical memory retrieval, self-referential thought, and social cognition. The DMN has been implicated in various cognitive functions including inference, active maintenance of information, and intertemporal choice. Additionally, alterations within the DMN have been observed in neurodevelopmental disorders like autism spectrum disorder (ASD) and neuropsychological conditions like mild cognitive impairment.

### Publication 1: Information-theoretic analysis

This publication addresses the relative quality of information gained from resting-state fMRI versus task-based fMRI studies. It finds that data from a Theory-of-Mind task provides significantly more information on effective connectivity compared to resting-state data, with DMN regions being associated with rest and thus may reflect phys

## Example query for Brain Input

The titles and abstract most related to the brain will be passed to the LM.

The functions expects an already encoded brain input

In [13]:
# Load network atlases
networks = _load_networks()

In [14]:
networks_resampled = resample_networks_to_mask(networks)

Resampling networks: 100%|██████████| 152/152 [00:03<00:00, 48.44it/s] 


In [15]:
networks_embed = {}

for k, v in tqdm(networks_resampled.items(), total=len(networks_resampled)):
    networks_embed[k] = autoencoder.encoder(torch.from_numpy(masker.transform(v)))

  0%|          | 0/152 [00:00<?, ?it/s]

In [17]:
# Look for abstract and titles related to Auditory network
abstract, titles, _ = search_papers_from_brain(networks_embed["AUD"], show_titles=True)

Top matches:
1. The retention of simultaneous tones in auditory short-term memory: A magnetoencephalography study
2. Spectral and temporal processing in human auditory cortex.
3. Auditory perception of natural sound categories – An fMRI study
4. Representation of the temporal envelope of sounds in the human brain.
5. From Vivaldi to Beatles and back: Predicting lateralized brain responses to music


In [ ]:
# LLM summary for related abstracts
output = generate_llm_response_from_brain(networks_embed["AUD"])

In [20]:
print(output)

The input brain activation pattern associated with the neuroscience concepts and cognitive terms provided indicates a complex interplay between auditory processing mechanisms and music cognition. The core areas of interest are temporal masking and melodic expectation, which highlight fundamental aspects of hearing and musical perception respectively. 

**Temporal Masking:**
- **Neuroscience Concepts:** Temporal masking involves auditory systems' frequency analysis capabilities and its relationship to the auditory bandwidths measured from forward masking thresholds. This suggests that the brain pattern is likely related to how the auditory system processes complex harmonic tones, particularly their fundamental frequencies.
- **Cognitive Atlas Terms:** Auditory stream segregation is a key cognitive function here as it involves grouping sounds to form coherent representations in the acoustic scene—a fundamental aspect of hearing and speech perception.

**Melodic Expectation:**
- **Neurosc